# BIGDATA FOR DYNAMICID

## Clone DynamicID Repository
- Gốc: https://github.com/traananhdat/DynamicID
- Mới: https://github.com/LongTruongPhamHai/bigdata-for-dynamicid

In [ ]:
# Xóa thư mục cũ nếu đã tồn tại
!rm -rf /kaggle/working/bigdata-for-dynamicid

# Clone repo github
!git clone https://github.com/LongTruongPhamHai/bigdata-for-dynamicid

## Train SAA

In [ ]:
# Tải thư viện
!pip uninstall -y diffusers huggingface_hub transformers accelerate peft

!pip install \
  diffusers==0.27.2 \
  transformers==4.41.2 \
  accelerate==0.27.2 \
  peft==0.10.0 \
  huggingface_hub==0.23.4

In [ ]:
# Kiểm tra thư viện
!pip show diffusers transformers accelerate peft

In [ ]:
!accelerate launch \
  --num_processes 1 \
  /kaggle/working/bigdata-for-dynamicid/train_SAA.py \
  --pretrained_model_name_or_path runwayml/stable-diffusion-v1-5 \
  --image_encoder_path openai/clip-vit-large-patch14 \
  --data_root_path /kaggle/input/dataset-for-dynamicid/dataset/base_image_dataset \
  --data_json_file /kaggle/working/bigdata-for-dynamicid/data.json \
  --mixed_precision fp16 \
  --train_batch_size 1 \
  --num_train_epochs 10

In [3]:
# Nén file checkpoint
!rm /kaggle/working/checkpoint-2000.zip
# !zip -r /kaggle/working/checkpoint-2000.zip /kaggle/working/sd-ip_adapter/checkpoint-2000

In [4]:
# Xóa 3 file checkpoint không cần thiết
!rm /kaggle/working/sd-ip_adapter/checkpoint-2000/optimizer.bin \
    /kaggle/working/sd-ip_adapter/checkpoint-2000/random_states_0.pkl \
    /kaggle/working/sd-ip_adapter/checkpoint-2000/scaler.pt

## Train IMR

### Tạo SAA.bin từ /sd-ip_adapter/checkpoint-2000/model.safetensors

In [ ]:
import torch
from safetensors.torch import load_file

ckpt_path = "sd-ip_adapter/checkpoint-2000/model.safetensors"
out_path = "models/SAA.bin"

state_dict = load_file(ckpt_path)

# chỉ giữ các key thuộc adapter / SAA
adapter_state = {
    k: v for k, v in state_dict.items()
    if "adapter" in k.lower() or "lora" in k.lower()
}

torch.save(adapter_state, out_path)
print("Exported SAA.bin")


## Cell cũ - Chỉ xem, không cần chạy

## Data Preprocess
### Rename image (Chạy trực tiếp trên máy)

In [ ]:
from pathlib import Path


def rename_images(directory: Path):
    files = [f for f in directory.iterdir() if f.is_file()]
    files.sort()  # sắp xếp để có thứ tự ổn định

    changed = 0
    for idx, f in enumerate(files, start=1):
        if idx > 35:
            break

        new_name = f"{idx:04d}.jpg"
        new_path = f.with_name(new_name)

        if new_path == f:
            continue  # đã đúng định dạng
        if new_path.exists():
            print(f"SKIP: Tồn tại sẵn {new_path.name}, bỏ qua {f.name}")
            continue

        f.rename(new_path)
        changed += 1
        print(f"RENAMED: {f.name} -> {new_path.name}")

    print(f"Done. Đã đổi tên {changed} file.")


if __name__ == "__main__":
    dir_path = Path(
        r"D:\DEV\.Projects\TLU\PTDL\DynamicID_fork_prv\dataset\base_image_dataset\2"
    ).resolve()
    rename_images(dir_path)


### FaceCrop Image (Đã xử lý và lưu vào dataset-for-dynamicid)
Model: MediaPipe Face Detection

In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.26.4 mediapipe==0.10.31 opencv-python

In [ ]:
!pip install mediapipe==0.10.31 opencv-python

In [ ]:
# Xóa thư mục nếu đã tồn tại
!rm -rf /kaggle/working/mediapipe_models

# Tạo thư mục mới
!mkdir -p /kaggle/working/mediapipe_models

# Tải model short-range
!wget \
https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/latest/blaze_face_short_range.tflite \
-O /kaggle/working/mediapipe_models/blaze_face_short_range.tflite

In [ ]:
import cv2
import os
from mediapipe import Image, ImageFormat
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# ===== PATH =====
base_input_dir = "/kaggle/input/dataset-for-dynamicid/dataset/base_image_dataset"
base_output_dir = "/kaggle/working/dataset/base_image_dataset"
short_model = "/kaggle/working/mediapipe_models/blaze_face_short_range.tflite"

def load_detector(model_path):
    base_options = python.BaseOptions(model_asset_path=model_path)
    options = vision.FaceDetectorOptions(
        base_options=base_options,
        running_mode=vision.RunningMode.IMAGE
    )
    return vision.FaceDetector.create_from_options(options)

# ===== LOAD MODEL =====
detector = load_detector(short_model)

# ===== PROCESS ALL SUBFOLDERS =====
for subfolder in sorted(os.listdir(base_input_dir)):
    input_dir = os.path.join(base_input_dir, subfolder)
    output_dir = os.path.join(base_output_dir, subfolder)

    if not os.path.isdir(input_dir):
        continue  # bỏ qua nếu không phải thư mục

    os.makedirs(output_dir, exist_ok=True)

    print(f"🔄 Processing folder: {subfolder}")

    for file in os.listdir(input_dir):
        if not file.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        img = cv2.imread(os.path.join(input_dir, file))
        if img is None:
            continue

        h, w, _ = img.shape
        mp_image = Image(
            image_format=ImageFormat.SRGB,
            data=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        )

        result = detector.detect(mp_image)

        if not result.detections:
            # ❌ Không nhận diện được mặt → lưu nguyên ảnh
            cv2.imwrite(os.path.join(output_dir, file), img)
            print(f"⚠️ No face detected, saved original: {file}")
            continue

        # lấy khuôn mặt đầu tiên
        bbox = result.detections[0].bounding_box
        x1, y1 = int(bbox.origin_x), int(bbox.origin_y)
        bw, bh = int(bbox.width), int(bbox.height)
        x2, y2 = x1 + bw, y1 + bh

        # mở rộng vùng crop vừa phải
        x_pad = int(0.2 * bw)
        y_pad = int(0.3 * bh)

        x1 = max(0, x1 - x_pad)
        y1 = max(0, y1 - y_pad)
        x2 = min(w, x2 + x_pad)
        y2 = min(h, y2 + y_pad)

        crop = img[y1:y2, x1:x2]
        cv2.imwrite(os.path.join(output_dir, file), crop)

        print(f"✔️ Cropped: {file}")

print("✅ DONE ALL FOLDERS")

In [ ]:
# Xóa file output.zip nếu đã tồn tại
!rm /kaggle/working/output.zip

# Nén toàn bộ thư mục output thành file output.zip
!zip -r /kaggle/working/output.zip /kaggle/working/dataset/base_image_dataset

### Tạo promt cho ảnh (Đã xử lý và lưu trong repo)
Model: BLIP

In [ ]:
!pip install transformers accelerate timm

In [ ]:
import os
import json
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import random

# Load BLIP model (chỉ để tham khảo, không dùng tên riêng)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")

root_path = "/kaggle/input/dataset-for-dynamicid/dataset/base_image_dataset"
data_list = []

# Danh sách biểu cảm và pose để sinh prompt mô tả khuôn mặt
expressions = [
    "smiling", "neutral expression", "serious face", "surprised look",
    "angry expression", "sad face", "laughing", "calm expression",
    "thoughtful look", "eyes closed", "wide-eyed", "gentle smile",
    "big grin", "pouting lips", "relaxed face", "focused eyes"
]

poses = [
    "facing forward", "looking to the left", "looking to the right",
    "head slightly tilted", "chin down", "chin up", "side profile",
    "three-quarter view", "looking upward", "looking downward"
]

def generate_face_prompt():
    expr = random.choice(expressions)
    pose = random.choice(poses)
    return f"{expr}, {pose}"

# Duyệt qua từng thư mục cá nhân (x = ID)
for person_id in sorted(os.listdir(root_path)):
    person_path = os.path.join(root_path, person_id)
    if not os.path.isdir(person_path):
        continue

    # Lấy danh sách ảnh trong thư mục
    img_files = [f for f in sorted(os.listdir(person_path)) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

    # Nếu nhiều hơn 35 → chỉ lấy 35 ảnh đầu
    img_files = img_files[:35]

    # Nếu ít hơn 35 → nhân bản cho đủ
    while len(img_files) < 35:
        img_files.append(img_files[-1])  # lặp lại ảnh cuối cùng

    # Tạo đúng 35 entry cho mỗi thư mục
    for idx, img_file in enumerate(img_files):
        img_path = os.path.join(person_path, img_file)
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            continue

        # Caption bằng BLIP (chỉ để tham khảo, không dùng tên riêng)
        try:
            inputs = processor(images=image, return_tensors="pt").to("cuda")
            out = model.generate(**inputs, max_new_tokens=50)
            caption_raw = processor.decode(out[0], skip_special_tokens=True)
        except Exception as e:
            print(f"BLIP failed on {img_path}: {e}")
            caption_raw = ""

        # Thay caption bằng prompt mô tả khuôn mặt (không tên riêng)
        caption = generate_face_prompt()

        rel_path = f"{person_id}/{img_file}"
        data_list.append({
            "image_file": rel_path,
            "text": caption
        })

# Xuất file JSON
output_json = "/kaggle/working/data.json"
with open(output_json, "w") as f:
    json.dump(data_list, f, indent=2)

print("Total entries created:", len(data_list))
print(f"Saved {len(data_list)} entries to {output_json}")

In [ ]:
# Sửa code train_SAA.py (Đã sửa lại trong repo gihub => Không cần chạy cell này)
from pathlib import Path

path = Path("/kaggle/working/DynamicID/train_SAA.py")
text = path.read_text()

text = text.replace(
    "ip_tokens = self.image_proj_model(image_embeds)",
    """dummy_id = torch.zeros(
        image_embeds.shape[0],
        512,
        device=image_embeds.device,
        dtype=image_embeds.dtype
    )
    ip_tokens = self.image_proj_model(dummy_id, image_embeds)"""
)

path.write_text(text)
print("✅ Fixed ProjPlusModel forward call")

In [ ]:
# Sửa code train_SAA.py (Đã sửa lại trong repo gihub => Không cần chạy cell này)
from pathlib import Path
import re

path = Path("/kaggle/working/DynamicID/train_SAA.py")
text = path.read_text()

pattern = re.compile(r"image_proj_model\s*=\s*ProjPlusModel\([\s\S]*?\)", re.MULTILINE)

replacement = """image_proj_model = ProjPlusModel(
        cross_attention_dim=unet.config.cross_attention_dim,
        id_embeddings_dim=512,
        clip_embeddings_dim=image_encoder.config.hidden_size,
        num_tokens=args.num_tokens,
    )"""

text = pattern.sub(replacement, text)
path.write_text(text)

print("✅ Fixed ProjPlusModel init")

In [ ]:
# Sửa code train_SAA.py (Đã sửa lại trong repo gihub => Không cần chạy cell này)
from pathlib import Path

file_path = Path("/kaggle/working/DynamicID/train_SAA.py")
text = file_path.read_text()

old = "attn_procs[name] = SAProcessor()"

new = """attn_procs[name] = SAProcessor(
                hidden_size=hidden_size,
                rank=4,
                network_alpha=4,
            )"""

text = text.replace(old, new)
file_path.write_text(text)

print("✅ Fixed SAProcessor init")

In [ ]:
# Sửa code train_SAA.py (Đã sửa lại trong repo gihub => Không cần chạy cell này)
from pathlib import Path

file_path = Path("/kaggle/working/DynamicID/train_SAA.py")
text = file_path.read_text()

text = text.replace(
    "attn_procs[name].load_state_dict(weights)",
    "attn_procs[name].load_state_dict(weights, strict=False)"
)

file_path.write_text(text)
print("✅ Fixed load_state_dict(strict=False)")